In [1]:
import sys 
sys.path.append('../../..')
from main_multitask_multimodal import LLM_MultitaskMultimodal

In [2]:
import json

In [3]:
model = LLM_MultitaskMultimodal.load_from_checkpoint('../../../models/distill_query_classify/version_6/epoch=2-step=604.ckpt'
                                                     ).cuda().eval()

INFO:root:Unused kwargs when getting microsoft/Multilingual-MiniLM-L12-H384: {'distance_func': 'cosine', 'loss_type': 'cross-entropy', 'margin': None, 'hidden_states_type': 'encoder-first', 'add_simcse': False, 'manual_loss_type': 'manual_mse', 'auto_task_weight': False, 'multitask_specs_dict': {'seqclf_singlemodal_wishquery2tax': {'specs': {'clf_head': 'clf_head', 'clf_weight_type': None, 'label_map_file': '/workspaces/multitask-llm-rnd/modelling/datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt', 'label_type': 'multilabel_taxonomy'}}}, 'head_dict': {'clf_head': {'purpose': 'seqclf', 'type': 'linear', 'in_features': 384, 'out_features': 6037}}}


# qualitative check

In [4]:
inputs = model.tokenizer("Classify query: dildo", return_tensors='pt').to('cuda')

In [5]:
probs = model.clf_head(model.get_hidden_states(**inputs)).sigmoid()

In [6]:
top_probs, top_preds = probs.topk(10)

In [7]:
top_preds

tensor([[1130, 1121, 1146, 1118, 1125, 1139, 1149, 1116, 1140, 1126]],
       device='cuda:0')

In [8]:
label_map_file = model.hparams.multitask_specs_dict['seqclf_singlemodal_wishquery2tax']['specs']['label_map_file']

In [9]:
label_map = {}
with open(label_map_file, 'r') as f:
    for l in f:
        l = l.replace('\n', '').strip()
        if len(l):
            label_map[l] = len(label_map)

In [10]:
label_map_rev = {label_map[i]: i for i in label_map}

In [11]:
[label_map_rev[i.item()] for i in top_preds[0]]

['beauty & health > sexual wellness > sex toys > dildos',
 'beauty & health > sexual wellness > sex toys',
 'beauty & health > sexual wellness > sex toys > vibrators',
 'beauty & health > sexual wellness > safer sex > condoms',
 'beauty & health > sexual wellness > sex toys > anal plug',
 'beauty & health > sexual wellness > sex toys > penis rings',
 'beauty & health > sexual wellness > sm products > bondage gear',
 'beauty & health > sexual wellness',
 'beauty & health > sexual wellness > sex toys > pumps & enlargers',
 'beauty & health > sexual wellness > sex toys > anal sex toys']

In [12]:
top_probs

tensor([[9.9035e-01, 2.5334e-02, 1.3481e-03, 8.7281e-04, 5.0182e-04, 3.0553e-04,
         3.0020e-04, 2.9364e-04, 2.1401e-04, 2.0970e-04]], device='cuda:0',
       grad_fn=<TopkBackward0>)

In [15]:
def infer_model(query, topk = 3):
    assert topk <= len(label_map_rev)
    inputs = model.tokenizer(f"Classify query: {query}", return_tensors='pt').to('cuda')
    probs = model.clf_head(model.get_hidden_states(**inputs)).sigmoid()
    top_probs, top_preds = probs.topk(topk)
    return [(label_map_rev[top_preds[0][ind].item()], top_probs[0][ind].item()) for ind in range(topk)]

In [16]:
infer_model('apple', 3)

[('cellphones & telecommunications > iphones', 0.6980677247047424),
 ('consumer electronics > smart electronics > wearable devices > smart watches',
  0.32453083992004395),
 ('computer & office > tablets', 0.09000036120414734)]

In [17]:
infer_model('iphone', 3)

[('cellphones & telecommunications > iphones', 0.9638227224349976),
 ('cellphones & telecommunications > mobile phone accessories',
  0.12202276289463043),
 ('cellphones & telecommunications > cellphones', 0.06417359411716461)]

In [18]:
badsearch = [
    "chopsticks",
    "keyboard for mac",
    "Logitech K380",
    "K380",
    "keyboard with usb dongle",
    "rc cars 1/10 scale brushless"
]

In [19]:
for q in badsearch:
    print(f" > {q}")
    print(infer_model(q.lower().strip(), 3))

 > chopsticks
[('home & garden > kitchen,dining & bar > flatware > chopsticks', 0.9893905520439148), ('home & garden > kitchen,dining & bar > disposable kitchenware > disposable chopsticks', 0.25948646664619446), ('beauty & health > beauty essentials > lipstick', 0.012643793597817421)]
 > keyboard for mac
[('computer & office > mouse & keyboards > keyboards', 0.9168869853019714), ('computer & office > mouse & keyboards > keyboard mouse combos', 0.10149267315864563), ('computer & office > laptop accessories > keyboard covers', 0.08296918123960495)]
 > Logitech K380
[('computer & office > mouse & keyboards > mice', 0.8567105531692505), ('computer & office > mouse & keyboards > mice & keyboards accessories', 0.12367086857557297), ('computer & office > laptop accessories > laptop mice', 0.06955510377883911)]
 > K380
[('security & protection > video surveillance > surveillance cameras', 0.17515814304351807), ('security & protection > self defense supplies', 0.035376228392124176), ('security

In [21]:
infer_model('Logitech K380'), infer_model('Logitech K380'.lower())

([('sports > hunting > hunting gun accessories', 0.22011637687683105),
  ('sports > hunting', 0.10330711305141449),
  ('sports > hunting > hunting optics > riflescopes', 0.036959145218133926)],
 [('computer & office > mouse & keyboards > mice', 0.8567105531692505),
  ('computer & office > mouse & keyboards > mice & keyboards accessories',
   0.12367086857557297),
  ('computer & office > laptop accessories > laptop mice',
   0.06955510377883911)])

# convert

In [22]:
import pandas as pd 
import dvc.api
from torch import nn
import torch

In [23]:
df_tax = pd.read_json(dvc.api.get_url(
    'datasets/data/taxonomy/wish_newtax.json',
    repo='git@github.com:ContextLogic/multitask-llm-rnd.git'
), lines=True)

INFO:aiobotocore.credentials:Found credentials in environment variables.


In [24]:
orig_cat2id = {}
lower2orig = {}
for i in df_tax.to_dict('records'):
    if len(i['category_path']) > 0:
        orig_cat2id[i['category_path'].lower()] = int(i['id'])
        lower2orig[i['category_path'].lower()] = i['category_path']

In [25]:
modelid2origid = {}
for i in label_map_rev:
    modelid2origid[i] = (orig_cat2id[label_map_rev[i]], lower2orig[label_map_rev[i]])
assert max(modelid2origid) == len(modelid2origid) - 1 and min(modelid2origid) == 0

In [26]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

In [28]:
model_hf = AutoModelForSequenceClassification.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384", 
                                                              id2label=modelid2origid).cuda().eval()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at microsoft/Multilingual-MiniLM-L12-H384 and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
tokenizer_hf = AutoTokenizer.from_pretrained("microsoft/Multilingual-MiniLM-L12-H384", fast=True)

In [30]:
set(model.transformer.state_dict().keys()) == set(model_hf.bert.state_dict().keys())

True

In [31]:
model_hf.bert.load_state_dict(model.transformer.state_dict())

<All keys matched successfully>

In [32]:
set(model.clf_head[1].state_dict().keys()) == set(model_hf.classifier.state_dict().keys())

True

In [33]:
model_hf.classifier.load_state_dict(model.clf_head[1].state_dict())

<All keys matched successfully>

In [40]:
with torch.no_grad():
    model_hf.bert.pooler.dense.weight = nn.Parameter(torch.eye(384).cuda())
    model_hf.bert.pooler.activation = nn.Identity()
    model_hf.bert.pooler.dense.bias = nn.Parameter(model_hf.bert.pooler.dense.bias * 0)

In [45]:
classify = pipeline("text-classification", model=model_hf,
        tokenizer=tokenizer_hf, function_to_apply='sigmoid', device=0)

In [46]:
classify('Classify query: apple', top_k=3)

[{'label': (1316, 'Cellphones & Telecommunications > iPhones'),
  'score': 0.6980677247047424},
 {'label': (1752,
   'Consumer Electronics > Smart Electronics > Wearable Devices > Smart Watches'),
  'score': 0.32453083992004395},
 {'label': (1494, 'Computer & Office > Tablets'),
  'score': 0.09000036120414734}]

In [47]:
infer_model('apple', 3)

[('cellphones & telecommunications > iphones', 0.6980677247047424),
 ('consumer electronics > smart electronics > wearable devices > smart watches',
  0.32453083992004395),
 ('computer & office > tablets', 0.09000036120414734)]

In [50]:
def infer_model_hf(query, topk = 3):
    assert topk <= len(label_map_rev)
    inputs = tokenizer_hf(f"Classify query: {query}", return_tensors='pt').to('cuda')
    probs = model_hf(**inputs).logits.sigmoid()
    top_probs, top_preds = probs.topk(topk)
    return [(label_map_rev[top_preds[0][ind].item()], top_probs[0][ind].item()) for ind in range(topk)]

In [51]:
infer_model_hf('apple', 3)

[('cellphones & telecommunications > iphones', 0.6980677247047424),
 ('consumer electronics > smart electronics > wearable devices > smart watches',
  0.32453083992004395),
 ('computer & office > tablets', 0.09000036120414734)]

In [52]:
%%timeit
classify('Classify query: apple', top_k=3)

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


17.5 ms ± 61.6 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [53]:
%%timeit
infer_model('apple', 3)

11 ms ± 37 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [54]:
%%timeit
infer_model_hf('apple', 3)

11.1 ms ± 42 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [55]:
infer_model_hf('apple', 3)

[('cellphones & telecommunications > iphones', 0.6980677247047424),
 ('consumer electronics > smart electronics > wearable devices > smart watches',
  0.32453083992004395),
 ('computer & office > tablets', 0.09000036120414734)]

In [56]:
for q in badsearch:
    b = json.dumps(infer_model_hf('apple', 3))
    c = json.dumps(infer_model('apple', 3))
    assert b==c

In [78]:
model_hf.save_pretrained('hf_ckpt')

In [79]:
tokenizer_hf.save_pretrained('hf_ckpt')

('hf_ckpt/tokenizer_config.json',
 'hf_ckpt/special_tokens_map.json',
 'hf_ckpt/sentencepiece.bpe.model',
 'hf_ckpt/added_tokens.json',
 'hf_ckpt/tokenizer.json')

In [57]:
model_hf_reload = AutoModelForSequenceClassification.from_pretrained('hf_ckpt').cuda().eval()

In [58]:
tokenizer_hf_reload = AutoTokenizer.from_pretrained('hf_ckpt')

In [59]:
def infer_model_hf_reload(query, topk = 3):
    assert topk <= len(label_map_rev)
    inputs = tokenizer_hf_reload(f"Classify query: {query}", return_tensors='pt').to('cuda')
    probs = model_hf_reload(**inputs).logits.sigmoid()
    top_probs, top_preds = probs.topk(topk)
    return [(label_map_rev[top_preds[0][ind].item()], top_probs[0][ind].item()) for ind in range(topk)]

In [60]:
infer_model_hf_reload('apple')

[('cellphones & telecommunications > iphones', 0.5591620802879333),
 ('consumer electronics > smart electronics > wearable devices > smart watches',
  0.2996892035007477),
 ('computer & office > tablets', 0.1604585498571396)]

In [61]:
with torch.no_grad():
    model_hf_reload.bert.pooler.activation = nn.Identity()

In [62]:
infer_model_hf_reload('apple')

[('cellphones & telecommunications > iphones', 0.6980677247047424),
 ('consumer electronics > smart electronics > wearable devices > smart watches',
  0.32453083992004395),
 ('computer & office > tablets', 0.09000036120414734)]

In [63]:
for q in badsearch:
    b = json.dumps(infer_model_hf('apple', 3))
    c = json.dumps(infer_model('apple', 3))
    d = json.dumps(infer_model_hf_reload('apple', 3))
    assert b==c==d

In [64]:
from hf_ckpt.load_model import load_pipeline

In [65]:
classify_reload = load_pipeline()

In [88]:
for q in badsearch:
    a = classify(f"Classify query: {q}")
    b = classify_reload(f"Classify query: {q}")
    c = infer_model(q)
    d = infer_model_hf(q)
    e = infer_model_hf_reload(q)
    assert json.dumps(a)==json.dumps(b)
    assert c[0][1] == d[0][1] == e[0][1] == a[0]['score'] == b[0]['score']

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
